## mysqlclient 설치

In [1]:
!pip install mysqlclient

mysqlclient를 설치했으므로 파이썬을 사용해서 데이터베이스에 접속해봅니다. MySQLdb를 import하고, create 함수의 매개 변수에 다음과 같은 것들을 지정해서 호출하면 데이터베이스에 접속됩니다. 

- 사용자 이름
- 비밀번호
- 호스트(localhost라면 생략 가능)
- 데이터베이스 이름
- 문자코드(생략하면 디폴트 문자 코드가 자동으로 지정됩니다.)


In [8]:
## 데이터베이스에 접속해서 SQL 실행하기

import MySQLdb

connection = MySQLdb.connect(
                            user='scrapingman',
                            passwd='myPassword-1',
                            host='localhost',
                            db='scrapingdata',
                            charset='utf8')

In [9]:
# connect 함수의 반환값은 Connection 클래스입니다.
type(connection)

MySQLdb.connections.Connection

In [10]:
# Connection 클래스의 cursor 메서드를 호출하면 커서를 추출할 수 있습니다.
cursor = connection.cursor()
type(cursor)

MySQLdb.cursors.Cursor

## 테이블 만들기

Cursor 클래스의 execute 메서드로 SQL을 실행합니다. 

In [11]:
cursor.execute('create table books (title text, url text)')

0

In [12]:
connection.commit()

```
mysql> show tables from scrapingdata;
+------------------------+
| Tables_in_scrapingdata |
+------------------------+
| books                  |
+------------------------+
1 row in set (0.05 sec)
```

```
mysql> show columns from books from scrapingdata;
+-------+------+------+-----+---------+-------+
| Field | Type | Null | Key | Default | Extra |
+-------+------+------+-----+---------+-------+
| title | text | YES  |     | NULL    |       |
| url   | text | YES  |     | NULL    |       |
+-------+------+------+-----+---------+-------+
2 rows in set (0.01 sec)
```

파이썬에서 데이터베이스에 접속합니다. 레코드를 추가할 떄는 Cursor 클래스의 execute 메서드를 사용합니다.    
Cursor클래스의 execute 메서드는 다음 코드처럼 두 번째 매개 변수를 지정할 수 있습니다.

In [16]:
import MySQLdb

connection = MySQLdb.connect(
                            user='scrapingman',
                            passwd='myPassword-1',
                            host='localhost',
                            db='scrapingdata',
                            charset='utf8')

cursor = connection.cursor()

# 실행할 때마다 다른 결과가 나오지 않게 테이블을 제거해두기
cursor.execute('drop table if exists books')

cursor.execute('create table books (title text, url text)')
connection.commit()
cursor.execute('insert into books values(%s, %s)', ('처음 시작하는 파이썬 프로그래밍', 'http://example.com'))
connection.commit()

# 연결 종료하기
connection.close()

1

# 분석 결과 저장하기

In [18]:
import feedparser
import MySQLdb

# 데이터베이스에 연결
connection = MySQLdb.connect(
                            user='scrapingman',
                            passwd='myPassword-1',
                            host='localhost',
                            db='scrapingdata',
                            charset='utf8')

# 커서 생성 
cursor = connection.cursor()

# 실행할 떄마다 같은 레코드가 중복되어 들어가지 않게 테이블을 제거해두기
cursor.execute('drop table if exists books')

# 테이블 생성하기
cursor.execute('create table books (title text, url text)')

# url을 지정해서 FeedParserDict 객체 생성하기
rss = feedparser.parse('http://www.aladin.co.kr/rss/special_new/351')

# rss version check
print(rss.version)

# 피드의 제목
print(rss.get('feed').get('title'))

# 반복 적용
for content in rss.get('entries'):
    #데이터 저장하기
    cursor.execute('insert into books values(%s, %s)', (content.get('title'), content.get('link')))

# 커밋하기
connection.commit()

# 연결 종료하기
connection.close()



rss20
분야별 신간 특선 - 컴퓨터/모바일


```
mysql> select * from books;
+--------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
| title                                                                                            | url                                                                            |
+--------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
| 코딩을 몰라도 아무나 웹사이트 가이드편/조안 지음/F&A                                             | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=253989653 |
| 아무나 웹사이트 회사일반 편/조안 지음/F&A                                                        | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=253989630 |
| 아무나 웹사이트 이커머스 편/조안 지음/F&A                                                        | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=253989610 |
| 아무나 웹사이트 오픈몰편/조안 지음/F&A                                                           | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=253989451 |
| MagicMirror HowTo(흑백)/소혜민 지음/부크크(bookk)                                                | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=253719046 |
| MagicMirror HowTo(컬러)/소혜민 지음/부크크(bookk)                                                | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=253718914 |
| 시작해 C 언어 핵심기초 (스터디북)/박영진 지음/부크크(bookk)                                      | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=253718005 |
| 따라하며 배우는 AWS 네트워크 입문/김원일.서종호 지음/부크크(bookk)                               | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=253645476 |
| 사전처럼 쏙쏙 뽑아 쓰는 엑셀 함수 85선 & 엑셀 분석 작업의 모든 것/김은희 지음/부크크(bookk)      | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=251848878 |
| 소프트웨어 공학/조민호 지음/인피니티북스                                                         | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=250625329 |
| 하루에 쪼갠다 파이썬/코딩 코치스 지음/부크크(bookk)                                              | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=250624769 |
| 알면 편리한 워드프로세서 파워 업/박미현.박성호 지음/부산대학교출판문화원                         | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=250552773 |
| 하루에 쪼갠다 C++ 알고리즘/박영진 지음/부크크(bookk)                                             | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=250458870 |
| NX(UG) CAD/CAM 5축 가공기술/윤일우.황종대 지음/광문각                                            | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=250221431 |
| 데이터분석을 위한 기계학습/올리버 테오발드 지음, 하성호 옮김/청람                                | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=250187152 |
| 한글 SPSS 통계자료분석/이기성.한성실.정기문 지음/자유아카데미                                    | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=250110224 |
| 예제로 쉽게 배우는 파이썬 프로그래밍/김상근 지음/다올미디어                                      | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=250043106 |
| 전자상거래관리사 2급 실기/신미영 지음/디지털북스(아이생각)                                       | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=249313598 |
| 따라 하면서 배우는 사물인터넷/양재삼 지음/한빛아카데미(교재)                                     | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=246889810 |
| 3D 프린팅 모델링 자격 2급 필기.실기 한 권에 끝내기/김보영 지음/원교재사                          | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=246550812 |
| 스스로 ISMS-P 수험서/유영일 외 지음/부크크(bookk)                                                | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=246519075 |
| ISMS-P 인증 한권으로 마스터하기/권성호 지음/부크크(bookk)                                        | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=246280483 |
| 국/가/공/인 산업보안관리사 한권으로 끝내기/최상용.조석상 지음/시대고시기획                       | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=244929058 |
| 3D 프린팅 모델링 자격 1급 필기.실기 한 권에 끝내기/김주화 지음/원교재사                          | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=244891216 |
| 3D프린터운용기능사 실기 30시간 완성 《인벤터편, 컬러본》/신동진.조예찬.김경미 지음/부크크(bookk) | http://www.aladin.co.kr/rsscenter/go.aspx?rssType=1&type=item&itemId=244280996 |
+--------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
25 rows in set (0.00 sec)
```